In [6]:
from urllib.request import urlopen  
from tqdm import tqdm
from lxml import html
from datetime import datetime
from collections import Counter
import json
import requests
import time
import re
# from screeninfo import get_monitors
import pandas as pd
import random
import os
import ast

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# US

In [7]:
#replace as needed
folder_name = "Supplementary Materials"
followers_path = os.path.join(folder_name, "followers_US.txt")
completed_path = os.path.join(folder_name, "completed_accounts")

followers_US = []
# with open(r'''X\followers_US.txt''', 'r') as f: # PC
with open(followers_path,'r') as f: #List of accounts from raw followers list
    for line in f:
        if "@" in line:
            follower = line.strip()[1:]
            if follower not in followers_US:
                # print(follower)
                followers_US.append(follower)
f.close()

with open(completed_path, 'r') as f: #Get list of accounts that are completed
    for line in f:
        if line.startswith('total'):
            completed_list = ast.literal_eval(line[8:])

#Filter out the ones that are completed
# followers_US_set = set(followers_US)
# followers_US = [f for f in followers_US_set if f not in completed_list]

print(followers_US)

print(f"Number of followers to process: {len(followers_US)}")

['PatTheSocialist', 'SabbySabs2', 'TristanSnell', 'BernardKerik', 'spann', 'jeff_poor', 'DavidAFrench', 'NancyAFrench', 'BillOReilly', 'Jaybefaunt', 'donwinslow', 'KyleClark', 'BobMurphyEcon', 'ThomasEWoods', 'bshelburne', 'PatrickSvitek', 'evanasmith', 'MsLaToshaBrown', 'TomFitton', 'BarbMcQuade', 'JohnArchibald', 'brad_polumbo', 'natsechobbyist', 'jimmy_dore', 'RonFilipkowski', 'davidhogg111', 'anthonyzenkus', 'benshapiro', 'IngrahamAngle', 'RealSpikeCohen', 'briebriejoy', 'ComicDaveSmith', 'TheRickWilson', 'FreemansMind96', 'HannahDCox', 'KyleKulinski', 'seanhannity', 'funder', 'fred_guttenberg', 'LeeCamp', 'JordanChariton', 'mmpadellan', 'charliekirk11', 'davidsirota', 'kylegriffin1', 'TuckerCarlson', 'elonmusk', 'itsJeffTiedrich', 'bluestein', 'krystalball', 'SarcasmStardust', 'AnnCoulter', 'SteveSchmidtSES', 'SocialistMMA', 'gtconway3d', 'ProudSocialist', 'checkmatestate', 'Josh_Moon', 'EricTrump', 'LaraLeaTrump', 'WarOnDumb', 'robreiner', 'marceelias', 'toddcstacy']
Number of fo

In [8]:
tweets_dict_US = {}

# Function to check if the Fetch More button is present
def is_element_present(driver, xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        return element.is_displayed()
    except NoSuchElementException:
        return False

In [9]:
driver = webdriver.Chrome()
driver.get('https://twitter.com/i/flow/login')
wait = WebDriverWait(driver, 10)


The chromedriver version (128.0.6613.137) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (129.0.6668.90); currently, chromedriver 129.0.6668.89 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


In [10]:
### Log in

# Username
username = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input[autocomplete=username]'))
)
username.send_keys("blaketrn@udel.edu")
time.sleep(3)
login_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '[role=button].r-13qz1uu'))
)
login_button.click()
time.sleep(3)

try: 
    # Password
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[type=password]'))
    )
    password.send_keys("Idontevenknow1!")
    time.sleep(3)
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid*=Login_Button]'))
    )
    login_button.click()
    time.sleep(3)
    
except: # if we get an error
    username = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextTextInput]"))
    )
    username.send_keys("HaHoangNha55719")
    time.sleep(3)
    
    next_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=ocfEnterTextNextButton]"))
    )
    next_button.click()
    time.sleep(3)
    
    password = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[name*=password]"))
    )
    password.send_keys("Idontevenknow1!")
    time.sleep(3)
    
    login_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid*=LoginForm_Login_Button]"))
    )
    login_button.click()
    time.sleep(3)
    

In [11]:
count = 1

keyword1 = "(Gaza* OR Israel* OR (West Bank) OR (Tel Aviv) OR (Tel Aviv-Yafo) OR Bedouin* OR Palestin* OR Middle OR East* OR Middle OR Eastern OR Jew* OR rabbi OR Muslim* OR Islam* OR Jihad OR Antisemiti* OR Zionis* OR IDF)"
keyword2 = "(IOF OR Hamas OR Massacre OR Genocide OR Ceasefire OR Terroris* OR Netanyahu OR Histadrut OR Haniyeh OR Yahya OR Sinwar OR Fatah OR Mohammed OR Deif)"
hashtags = "(# OR #IsraelPalestineWar OR #IsraelGazaWar OR #Gazabombing OR #CeasefireNOW OR #FreePalestine OR #StrikeForGaza OR #Gazagenocide)"
until = "2023-10-07"
since = "2022-10-07"

unprocessed = []

for profile in tqdm(followers_US):
    if count%5 == 0:
        print("5 Accounts passed, sleeping for 7 minutes.")
        time.sleep(420)
    
    if profile not in tweets_dict_US.keys():
        print(profile)

        tweets_dict_US[profile] = []

        ### Begin scrape
        time.sleep(3)

        # Search
        search_button = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search and explore"]'))
        )
        search_button.click()
        time.sleep(3)

        search_bar = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search"]'))
        )
        search_bar.click()
        time.sleep(3)

        for i in range(2): 
            search_query = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Search query"]')))

            # Advanced search query
            if i == 0:
                search_query.send_keys(f'{keyword1} {hashtags} (from:{profile}) min_faves:1000 until:{until} since:{since} -filter:links -filter:replies')
            if i == 1:
                search_query.send_keys([Keys.BACKSPACE]*1000)
                search_query.send_keys(f'{keyword2} {hashtags} (from:{profile}) min_faves:1000 until:{until} since:{since} -filter:links -filter:replies')

            search_query.send_keys(Keys.RETURN)
            time.sleep(6)
            
            # Scroll until you cannot scroll anymore
            reached_page_end = False
            last_height = driver.execute_script("return document.body.scrollHeight")

            retry_xpath = "//button[@role='button' and @type='button']//span[text()='Retry']"
            # random.randrange(10, 15)
            if (is_element_present(driver, retry_xpath)):
                try:
                    print(f'{profile} - {count} - Retry button appeared.')
                    unprocessed.append(profile)
                    for i in range(5):
                        search_button = wait.until(EC.element_to_be_clickable((By.XPATH, retry_xpath)))
                        search_button.click()
                    time.sleep(100) 
                except:
                    print(f"{profile} - Failed to click Retry")

            while not reached_page_end:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")   
                time.sleep(4)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if last_height == new_height:
                    reached_page_end = True
                else:
                    last_height = new_height
            try:
                # Select the home timeline div
                search_results = driver.find_element(By.CSS_SELECTOR, '[aria-label="Timeline: Search timeline"]')
                # Just get all the HTML, will parse later
                tweets_listed = search_results.get_attribute('innerHTML')
                time.sleep(3)

                # Append to dict
                tweets_dict_US[profile].append(tweets_listed)

            except:
                pass

    count+=1

print("Unprocessed accounts for this run:")
for profile in unprocessed:
    print(f"@{profile}\n")


  0%|          | 0/64 [00:00<?, ?it/s]

PatTheSocialist


  2%|▏         | 1/64 [00:42<44:31, 42.40s/it]

SabbySabs2


  3%|▎         | 2/64 [01:27<45:24, 43.95s/it]

...

FreemansMind96


 53%|█████▎    | 34/64 [1:09:24<40:12, 80.42s/it]

5 Accounts passed, sleeping for 7 minutes.
HannahDCox


 53%|█████▎    | 34/64 [1:35:41<1:24:26, 168.88s/it]


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=129.0.6668.90)
Stacktrace:
0   chromedriver                        0x0000000103d62338 chromedriver + 5096248
1   chromedriver                        0x0000000103d59b6a chromedriver + 5061482
2   chromedriver                        0x00000001038e4fd0 chromedriver + 389072
3   chromedriver                        0x00000001038cc466 chromedriver + 287846
4   chromedriver                        0x00000001038cc363 chromedriver + 287587
5   chromedriver                        0x00000001038e7302 chromedriver + 398082
6   chromedriver                        0x0000000103972c73 chromedriver + 969843
7   chromedriver                        0x0000000103955c93 chromedriver + 851091
8   chromedriver                        0x0000000103924c79 chromedriver + 650361
9   chromedriver                        0x000000010392549e chromedr

In [13]:
# driver.quit()

In [14]:
tweets_dict_US.keys()

dict_keys(['PatTheSocialist', 'SabbySabs2', 'TristanSnell', 'BernardKerik', 'spann', 'jeff_poor', 'DavidAFrench', 'NancyAFrench', 'BillOReilly', 'Jaybefaunt', 'donwinslow', 'KyleClark', 'BobMurphyEcon', 'ThomasEWoods', 'bshelburne', 'PatrickSvitek', 'evanasmith', 'MsLaToshaBrown', 'TomFitton', 'BarbMcQuade', 'JohnArchibald', 'brad_polumbo', 'natsechobbyist', 'jimmy_dore', 'RonFilipkowski', 'davidhogg111', 'anthonyzenkus', 'benshapiro', 'IngrahamAngle', 'RealSpikeCohen', 'briebriejoy', 'ComicDaveSmith', 'TheRickWilson', 'FreemansMind96', 'HannahDCox', 'KyleKulinski', 'seanhannity', 'funder', 'fred_guttenberg', 'LeeCamp', 'JordanChariton', 'mmpadellan', 'charliekirk11', 'davidsirota', 'kylegriffin1', 'TuckerCarlson', 'elonmusk', 'itsJeffTiedrich', 'bluestein', 'krystalball', 'SarcasmStardust', 'AnnCoulter', 'SteveSchmidtSES', 'SocialistMMA', 'gtconway3d', 'ProudSocialist', 'checkmatestate', 'Josh_Moon', 'EricTrump', 'LaraLeaTrump', 'WarOnDumb', 'robreiner', 'marceelias', 'toddcstacy'])

In [15]:
tweets_dict_US_cleaned = {}
    
for key, value in tweets_dict_US.items():
    if key not in tweets_dict_US_cleaned.keys():
        tweets_dict_US_cleaned[key] = []
        for v in value:
            soup = BeautifulSoup(v, 'html.parser')
            tweets = soup.find_all(attrs={"data-testid": "tweetText"})
            for tweet in tweets:
                tweet_text = tweet.get_text(strip=True)
                tweets_dict_US_cleaned[key].append(tweet_text)

tweets_dict_US_cleaned.keys()

dict_keys(['PatTheSocialist', 'SabbySabs2', 'TristanSnell', 'BernardKerik', 'spann', 'jeff_poor', 'DavidAFrench', 'NancyAFrench', 'BillOReilly', 'Jaybefaunt', 'donwinslow', 'KyleClark', 'BobMurphyEcon', 'ThomasEWoods', 'bshelburne', 'PatrickSvitek', 'evanasmith', 'MsLaToshaBrown', 'TomFitton', 'BarbMcQuade', 'JohnArchibald', 'brad_polumbo', 'natsechobbyist', 'jimmy_dore', 'RonFilipkowski', 'davidhogg111', 'anthonyzenkus', 'benshapiro', 'IngrahamAngle', 'RealSpikeCohen', 'briebriejoy', 'ComicDaveSmith', 'TheRickWilson', 'FreemansMind96', 'HannahDCox', 'KyleKulinski', 'seanhannity', 'funder', 'fred_guttenberg', 'LeeCamp', 'JordanChariton', 'mmpadellan', 'charliekirk11', 'davidsirota', 'kylegriffin1', 'TuckerCarlson', 'elonmusk', 'itsJeffTiedrich', 'bluestein', 'krystalball', 'SarcasmStardust', 'AnnCoulter', 'SteveSchmidtSES', 'SocialistMMA', 'gtconway3d', 'ProudSocialist', 'checkmatestate', 'Josh_Moon', 'EricTrump', 'LaraLeaTrump', 'WarOnDumb', 'robreiner', 'marceelias', 'toddcstacy'])

In [16]:
raw_data_folder = os.path.join("Raw Data", "Before")
raw_json = os.path.join(raw_data_folder, f"US_tweets_{until}_{since}.json")
cleaned_json = os.path.join(raw_data_folder, f"US_tweets_cleaned_{until}_{since}.json")

In [17]:
with open(raw_json, 'w') as file:
    json.dump(tweets_dict_US, file)

In [18]:
with open(cleaned_json, 'w') as file:
    json.dump(tweets_dict_US_cleaned, file)